<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/version2/prepare_data/covid_data/Load_CovidData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task:
* old: weekly_CovidCases_CA_NY.ipynb file;
* need cleaning; Remove everything except COVID data loading 

# Topic: Investigate Covid-19 NY and CA data

This colab includes CA and NY covid case data from JHU and moving avg trend analysis

In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=ce751eee25115a5dacebe9f4e795a78b541e2814d85edf02a1a791bc92cd5cea
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


## Import Libraries

In [ ]:
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import wget
from pandas import Series, datetime
from pandas.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import random
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from xgboost import XGBClassifier
from sklearn.mixture import GaussianMixture

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Mount Google drive


In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import warnings; warnings.simplefilter('ignore')

In [ ]:
from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
location = "drive/Shared drives/the-mean-sqaures/the-mean-squares/Colab_Dataset/Dataset/"

In [ ]:
!ls /content/drive/My\ Drive/MasterProject-Personal/data

ls: cannot access '/content/drive/My Drive/MasterProject-Personal/data': No such file or directory


# Data Load

## 1 Load County Population


In [ ]:
county_population_US = pd.read_csv('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv',low_memory=False)
print(county_population_US.shape)

(3195, 4)


In [ ]:
wget.download('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv')
county_population_US = pd.read_csv('covid_county_population_usafacts.csv',low_memory=False)
print(county_population_US.shape)




(3195, 4)


In [ ]:
county_population_US.head(2)

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869


## 2 Load Covid-19 case details (until July 12)

In [ ]:
!ls '/content/drive/My Drive/MasterProject-Personal/data/'

ls: cannot access '/content/drive/My Drive/MasterProject-Personal/data/': No such file or directory


### Note about data:
John Hopkins university updates data every day hence we are pulling from repository directly

**US Confirmed url** :https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv

**US deaths url**: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv

In [ ]:
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv']

[wget.download(url) for url in urls]        

['time_series_covid19_confirmed_US.csv', 'time_series_covid19_deaths_US.csv']

In [ ]:
confirmed_US = pd.read_csv('time_series_covid19_confirmed_US.csv',low_memory=False)
death_US = pd.read_csv('time_series_covid19_deaths_US.csv',low_memory=False)
print(confirmed_US.shape)
print(death_US.shape)
print(confirmed_US.head(2))
death_US.head(2)

(3340, 353)
(3340, 354)
        UID iso2 iso3  code3  ...  12/25/20 12/26/20 12/27/20 12/28/20
0  84001001   US  USA    840  ...      3990     3999     4029     4065
1  84001003   US  USA    840  ...     12666    12708    12825    12962

[2 rows x 353 columns]


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,...,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",55869,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,39,39,39,39,39,39,41,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,41,41,41,43,43,43,44,44,44,44,44,46,46,46,46,47,47
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",223234,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,84,84,84,84,84,84,98,98,98,98,98,98,137,137,137,138,138,138,138,141,141,141,141,141,141,141,141,145,145,146,147,147,147,149,151,151,151,151,152,152


In [ ]:
#print(len(mask_data['state_name'].unique()))
print(len(confirmed_US['Province_State'].unique()))
confirmed_US['Province_State'].unique()

58


array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'Diamond Princess', 'District of Columbia', 'Florida', 'Georgia',
       'Grand Princess', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Northern Mariana Islands', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virgin Islands', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

## Data cleaning

In [ ]:
confirmed_US.columns[:11]

# Note: the first 11 columns contains UID, ios2,ios3, code ,FIPS, 'Admin2', 'Province_State',
      # 'Country_Region', 'Lat', 'Long_',Combined_Key', 'Population',


Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key'],
      dtype='object')

In [ ]:
# date begins from 11th column
confirmed_dates = confirmed_US.columns[11:]
confirmed_dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '12/19/20', '12/20/20', '12/21/20', '12/22/20', '12/23/20', '12/24/20',
       '12/25/20', '12/26/20', '12/27/20', '12/28/20'],
      dtype='object', length=342)

In [ ]:
death_US.columns[:12]

Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population'],
      dtype='object')

In [ ]:
death_US.columns[10:]

Index(['Combined_Key', 'Population', '1/22/20', '1/23/20', '1/24/20',
       '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20',
       ...
       '12/19/20', '12/20/20', '12/21/20', '12/22/20', '12/23/20', '12/24/20',
       '12/25/20', '12/26/20', '12/27/20', '12/28/20'],
      dtype='object', length=344)

In [ ]:
death_dates = death_US.columns[12:]
death_dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '12/19/20', '12/20/20', '12/21/20', '12/22/20', '12/23/20', '12/24/20',
       '12/25/20', '12/26/20', '12/27/20', '12/28/20'],
      dtype='object', length=342)

#### Note: both the date values has the same beginning date 1/22/2020. Hence we can use either of the values

In [ ]:
confirmed_df_long = confirmed_US.melt(
    id_vars=['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key'],
       value_vars=confirmed_dates,
       var_name = 'Date',
       value_name = 'Confirmed'
)

death_df_long = death_US.melt(
    id_vars=['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population'],
       value_vars=death_dates,
       var_name = 'Date',
       value_name = 'Deaths'
)

In [ ]:
confirmed_df_long.tail(10)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed
1142270,84056029,US,USA,840,56029.0,Park,Wyoming,US,44.521575,-109.585282,"Park, Wyoming, US",12/28/20,1939
1142271,84056031,US,USA,840,56031.0,Platte,Wyoming,US,42.132991,-104.966331,"Platte, Wyoming, US",12/28/20,421
1142272,84056033,US,USA,840,56033.0,Sheridan,Wyoming,US,44.790489,-106.886239,"Sheridan, Wyoming, US",12/28/20,2491
1142273,84056035,US,USA,840,56035.0,Sublette,Wyoming,US,42.765583,-109.913092,"Sublette, Wyoming, US",12/28/20,586
1142274,84056037,US,USA,840,56037.0,Sweetwater,Wyoming,US,41.659439,-108.882788,"Sweetwater, Wyoming, US",12/28/20,2921
1142275,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",12/28/20,2108
1142276,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",12/28/20,1507
1142277,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,"Unassigned, Wyoming, US",12/28/20,0
1142278,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,"Washakie, Wyoming, US",12/28/20,763
1142279,84056045,US,USA,840,56045.0,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",12/28/20,467


In [ ]:
confirmed_df_long[confirmed_df_long['FIPS'] == 36081].tail(30)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed
1044021,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",11/29/20,89681
1047361,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",11/30/20,90398
1050701,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/1/20,91056
1054041,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/2/20,91955
1057381,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/3/20,92813
1060721,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/4/20,93723
1064061,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/5/20,94584
1067401,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/6/20,95446
1070741,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/7/20,96267
1074081,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/8/20,97147


In [ ]:
death_df_long.tail(10)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,Date,Deaths
1142270,84056029,US,USA,840,56029.0,Park,Wyoming,US,44.521575,-109.585283,"Park, Wyoming, US",29194,12/28/20,7
1142271,84056031,US,USA,840,56031.0,Platte,Wyoming,US,42.132991,-104.966331,"Platte, Wyoming, US",8393,12/28/20,8
1142272,84056033,US,USA,840,56033.0,Sheridan,Wyoming,US,44.790489,-106.886239,"Sheridan, Wyoming, US",30485,12/28/20,16
1142273,84056035,US,USA,840,56035.0,Sublette,Wyoming,US,42.765583,-109.913092,"Sublette, Wyoming, US",9831,12/28/20,5
1142274,84056037,US,USA,840,56037.0,Sweetwater,Wyoming,US,41.659439,-108.882788,"Sweetwater, Wyoming, US",42343,12/28/20,15
1142275,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",23464,12/28/20,3
1142276,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",20226,12/28/20,7
1142277,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,"Unassigned, Wyoming, US",0,12/28/20,0
1142278,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,"Washakie, Wyoming, US",7805,12/28/20,16
1142279,84056045,US,USA,840,56045.0,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",6927,12/28/20,2


## Check California-New York data

In [ ]:
state= ['California','New York']
confirmed_CA_df = confirmed_df_long[confirmed_df_long['Province_State'].isin(state)]
confirmed_CA_df.tail(5)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed
1140899,84036115,US,USA,840,36115.0,Washington,New York,US,43.311538,-73.430434,"Washington, New York, US",12/28/20,767
1140900,84036117,US,USA,840,36117.0,Wayne,New York,US,43.154944,-77.029765,"Wayne, New York, US",12/28/20,2352
1140901,84036119,US,USA,840,36119.0,Westchester,New York,US,41.162784,-73.757417,"Westchester, New York, US",12/28/20,67388
1140902,84036121,US,USA,840,36121.0,Wyoming,New York,US,42.701451,-78.221996,"Wyoming, New York, US",12/28/20,1253
1140903,84036123,US,USA,840,36123.0,Yates,New York,US,42.635055,-77.103699,"Yates, New York, US",12/28/20,462


In [ ]:
confirmed_CA_df.Province_State.unique()

array(['California', 'New York'], dtype=object)

## Merging Confirmed and Death data

In [ ]:
full_table = confirmed_df_long.merge(
    right=death_df_long,
    how='left',
    on=[ 'UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key','Date']
)

full_table.head(10)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Population,Deaths
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",1/22/20,0,55869.0,0.0
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",1/22/20,0,223234.0,0.0
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",1/22/20,0,24686.0,0.0
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",1/22/20,0,22394.0,0.0
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",1/22/20,0,57826.0,0.0
5,84001011,US,USA,840,1011.0,Bullock,Alabama,US,32.100305,-85.712655,"Bullock, Alabama, US",1/22/20,0,10101.0,0.0
6,84001013,US,USA,840,1013.0,Butler,Alabama,US,31.753001,-86.680575,"Butler, Alabama, US",1/22/20,0,19448.0,0.0
7,84001015,US,USA,840,1015.0,Calhoun,Alabama,US,33.774837,-85.826304,"Calhoun, Alabama, US",1/22/20,0,113605.0,0.0
8,84001017,US,USA,840,1017.0,Chambers,Alabama,US,32.913601,-85.390727,"Chambers, Alabama, US",1/22/20,0,33254.0,0.0
9,84001019,US,USA,840,1019.0,Cherokee,Alabama,US,34.178060,-85.606390,"Cherokee, Alabama, US",1/22/20,0,26196.0,0.0


In [ ]:
#full_table['Date'] = pd.to_datetime(full_table['Date'])


In [ ]:
ship_data = full_table['Province_State'].str.contains('Grand Princess') | full_table['Province_State'].str.contains('Diamond Princess') | full_table['Province_State'].str.contains('Northern Mariana Islands') | full_table['Province_State'].str.contains('American Samoa') |full_table['Province_State'].str.contains('Guam') | full_table['Province_State'].str.contains('Virgin Islands')

full_ship = full_table[ship_data]


In [ ]:
# Removing ship data from State data

full_table = full_table[~(ship_data)]

## Group data

In [ ]:
full_grouped = full_table.groupby(['Date', 'Province_State','FIPS'])['Confirmed', 'Deaths'].sum().reset_index()

full_grouped.tail(5)

,Date,Province_State,FIPS,Confirmed,Deaths
1136803,9/9/20,Wyoming,56041.0,308,2.0
1136804,9/9/20,Wyoming,56043.0,111,6.0
1136805,9/9/20,Wyoming,56045.0,21,0.0
1136806,9/9/20,Wyoming,80056.0,0,0.0
1136807,9/9/20,Wyoming,90056.0,0,0.0


In [ ]:
NY_full_grouped = full_grouped[full_grouped['Province_State'] == 'New York']
NY_full_grouped[NY_full_grouped['FIPS'] == 36081.0]

,Date,Province_State,FIPS,Confirmed,Deaths
1932,1/22/20,New York,36081.0,0,0.0
5256,1/23/20,New York,36081.0,0,0.0
8580,1/24/20,New York,36081.0,0,0.0
11904,1/25/20,New York,36081.0,0,0.0
15228,1/26/20,New York,36081.0,0,0.0
...,...,...,...,...,...
1122120,9/5/20,New York,36081.0,70549,7234.0
1125444,9/6/20,New York,36081.0,70625,7234.0
1128768,9/7/20,New York,36081.0,70675,7234.0
1132092,9/8/20,New York,36081.0,70766,7235.0


In [ ]:
full_grouped.shape

(1136808, 5)

####  Adding new cases, new deaths by subtracting from previous day record

In [ ]:
full_grouped_ = full_grouped.copy()

In [ ]:
full_grouped = full_grouped_.copy()

In [ ]:
def fixDate(x):
  arr = x.split('/')
  m = arr[0]
  d = arr[1]
  y = arr[2]

  if int(m) < 10:
    m = '0'+str(m)
  if int(d) < 10:
    d = '0'+str(d)
  return '20'+str(y)+'-'+m+'-'+d

In [ ]:
full_grouped['Date'] = full_grouped['Date'].apply(lambda x: fixDate(x))

full_grouped.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths
0,2020-01-22,Alabama,1001.0,0,0.0
1,2020-01-22,Alabama,1003.0,0,0.0


In [ ]:
def computeNewCases(FIPS):


  countyData = full_grouped[full_grouped['FIPS'] == FIPS]
  countyData = countyData.sort_values(by=['Date'])

  if countyData.Confirmed.max() > 0:
    temp = countyData.groupby(['Date'])['Confirmed', 'Deaths']
    temp = temp.sum().diff().reset_index()
    #print(temp)
    mask = temp['Date'] != temp['Date'].shift(1)

    #temp.loc[mask, 'Confirmed'] = np.nan
    #temp.loc[mask, 'Deaths'] = np.nan
    temp.columns = ['Date', 'New cases', 'New deaths']
    print(temp)
    countyData = pd.merge(countyData, temp, on=[ 'Date'])
    print(countyData)
    #print('********')
    # filling na with 0
    countyData = countyData.fillna(0)
    # fixing data types
    cols = ['New cases', 'New deaths']
    countyData[cols] = countyData[cols].astype('int') 
    return countyData


In [ ]:
computeNewCases(36041)

           Date  New cases  New deaths
0    2020-01-22        NaN         NaN
1    2020-01-23        0.0         0.0
2    2020-01-24        0.0         0.0
3    2020-01-25        0.0         0.0
4    2020-01-26        0.0         0.0
..          ...        ...         ...
337  2020-12-24        7.0         0.0
338  2020-12-25        0.0         0.0
339  2020-12-26        1.0         0.0
340  2020-12-27        0.0         0.0
341  2020-12-28        1.0         0.0

[342 rows x 3 columns]
           Date Province_State     FIPS  ...  Deaths  New cases  New deaths
0    2020-01-22       New York  36041.0  ...     0.0        NaN         NaN
1    2020-01-23       New York  36041.0  ...     0.0        0.0         0.0
2    2020-01-24       New York  36041.0  ...     0.0        0.0         0.0
3    2020-01-25       New York  36041.0  ...     0.0        0.0         0.0
4    2020-01-26       New York  36041.0  ...     0.0        0.0         0.0
..          ...            ...      ...  ...     ...

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
0,2020-01-22,New York,36041.0,0,0.0,0,0
1,2020-01-23,New York,36041.0,0,0.0,0,0
2,2020-01-24,New York,36041.0,0,0.0,0,0
3,2020-01-25,New York,36041.0,0,0.0,0,0
4,2020-01-26,New York,36041.0,0,0.0,0,0
...,...,...,...,...,...,...,...
337,2020-12-24,New York,36041.0,97,1.0,7,0
338,2020-12-25,New York,36041.0,97,1.0,0,0
339,2020-12-26,New York,36041.0,98,1.0,1,0
340,2020-12-27,New York,36041.0,98,1.0,0,0


Below code executes for all FIPS, estimated runtime **10 minutes**

In [ ]:
df = pd.DataFrame(columns=['Date','Province_State','FIPS','Confirmed','Deaths','New cases','New deaths'])

for fips in tqdm(full_grouped.FIPS.unique()):
  

  countyData = full_grouped[full_grouped['FIPS'] == fips]
  countyData = countyData.sort_values(by=['Date'])
  #print(fips,' , ',countyData.Confirmed.min())
  
  if countyData.Confirmed.max() > 0:
    temp = countyData.groupby(['Date'])['Confirmed', 'Deaths']
    temp = temp.sum().diff().reset_index()
    #print(temp)
    mask = temp['Date'] != temp['Date'].shift(1)

    # temp.loc[mask, 'Confirmed'] = np.nan
    # temp.loc[mask, 'Deaths'] = np.nan
    temp.columns = ['Date', 'New cases', 'New deaths']
    countyData = pd.merge(countyData, temp, on=[ 'Date'])
    #print(countyData)
    #print('********')
    # filling na with 0
    countyData = countyData.fillna(0)
    # fixing data types
    cols = ['New cases', 'New deaths']
    countyData[cols] = countyData[cols].astype('int')
    df = df.append(countyData)
    #break

100%|██████████| 3324/3324 [09:34<00:00,  5.78it/s]


In [ ]:
print(df.shape)
df.tail(30)

(1115262, 7)


,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
312,2020-11-29,Wyoming,90056.0,0,0.0,0,0
313,2020-11-30,Wyoming,90056.0,0,0.0,0,0
314,2020-12-01,Wyoming,90056.0,0,9.0,0,9
315,2020-12-02,Wyoming,90056.0,0,9.0,0,0
316,2020-12-03,Wyoming,90056.0,0,9.0,0,0
317,2020-12-04,Wyoming,90056.0,0,9.0,0,0
318,2020-12-05,Wyoming,90056.0,0,9.0,0,0
319,2020-12-06,Wyoming,90056.0,0,9.0,0,0
320,2020-12-07,Wyoming,90056.0,0,9.0,0,0
321,2020-12-08,Wyoming,90056.0,0,9.0,0,0


In [ ]:
df.shape

(1115262, 7)

In [ ]:
df[df['FIPS'] == 56041.0].tail(50)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
292,2020-11-09,Wyoming,56041.0,568,4.0,17,0
293,2020-11-10,Wyoming,56041.0,614,4.0,46,0
294,2020-11-11,Wyoming,56041.0,627,4.0,13,0
295,2020-11-12,Wyoming,56041.0,664,4.0,37,0
296,2020-11-13,Wyoming,56041.0,686,4.0,22,0
297,2020-11-14,Wyoming,56041.0,719,4.0,33,0
298,2020-11-15,Wyoming,56041.0,729,4.0,10,0
299,2020-11-16,Wyoming,56041.0,760,4.0,31,0
300,2020-11-17,Wyoming,56041.0,796,4.0,36,0
301,2020-11-18,Wyoming,56041.0,835,4.0,39,0


In [ ]:
# merging new values
full_grouped = df.copy()

In [ ]:
full_grouped.tail(5)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
337,2020-12-24,Wyoming,90056.0,0,0.0,0,0
338,2020-12-25,Wyoming,90056.0,0,0.0,0,0
339,2020-12-26,Wyoming,90056.0,0,0.0,0,0
340,2020-12-27,Wyoming,90056.0,0,0.0,0,0
341,2020-12-28,Wyoming,90056.0,0,0.0,0,0


In [ ]:
state= ['California','New York']
ca_df = full_grouped[full_grouped['Province_State'].isin(state)]
ca_df.tail(10)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
332,2020-12-19,New York,90036.0,0,53.0,0,0
333,2020-12-20,New York,90036.0,0,60.0,0,7
334,2020-12-21,New York,90036.0,0,62.0,0,2
335,2020-12-22,New York,90036.0,0,68.0,0,6
336,2020-12-23,New York,90036.0,0,58.0,0,-10
337,2020-12-24,New York,90036.0,0,66.0,0,8
338,2020-12-25,New York,90036.0,0,70.0,0,4
339,2020-12-26,New York,90036.0,0,82.0,0,12
340,2020-12-27,New York,90036.0,0,93.0,0,11
341,2020-12-28,New York,90036.0,0,93.0,0,0


In [ ]:
county_population_US.head(2)

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869


In [ ]:
county_population_US['FIPS'] = county_population_US['countyFIPS']

In [ ]:
ca_df.shape

(41724, 7)

# Merge with FIPS 

In [ ]:
merged = pd.merge(ca_df,county_population_US,how='inner' ,on=['FIPS'])
print(merged.shape)

(41040, 11)


In [ ]:
merged.tail(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,countyFIPS,County Name,State,population
41038,2020-12-27,New York,36123.0,450,10.0,3,0,36123,Yates County,NY,24913
41039,2020-12-28,New York,36123.0,462,10.0,12,0,36123,Yates County,NY,24913


## Visualization

In [ ]:
full_grouped.head(3)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
0,2020-01-22,Alabama,1001.0,0,0.0,0,0
1,2020-01-23,Alabama,1001.0,0,0.0,0,0
2,2020-01-24,Alabama,1001.0,0,0.0,0,0


In [ ]:
merged.head(3)


,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,countyFIPS,County Name,State,population
0,2020-01-22,California,6001.0,0,0.0,0,0,6001,Alameda County,CA,1671329
1,2020-01-23,California,6001.0,0,0.0,0,0,6001,Alameda County,CA,1671329
2,2020-01-24,California,6001.0,0,0.0,0,0,6001,Alameda County,CA,1671329


In [ ]:
import pandas as pd
import altair as alt
#full_grouped = merged
ca_df = full_grouped[full_grouped['Province_State'] == 'California']
ny_df = full_grouped[full_grouped['Province_State'] == 'New York']

ca_df.shape

(20178, 7)

In [ ]:
ca_total = (
    ca_df
    .pipe(lambda x: x.assign(gains_pctg=x["New cases"]))
    .groupby(['Date','Province_State'])
    .agg({"gains_pctg": "sum"})
    .reset_index()
    .rename(columns={"gains_pctg": "New cases"})
)
ny_total = (
    ny_df
    .pipe(lambda x: x.assign(gains_pctg=x["New cases"]))
    .groupby(['Date','Province_State'])
    .agg({"gains_pctg": "sum"})
    .reset_index()
    .rename(columns={"gains_pctg": "New cases"})
)



In [ ]:
base_ca = alt.Chart(ca_total).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)

base_ny = alt.Chart(ny_total).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)



In [ ]:
red = alt.value("#f54242")
##Ca data
#base_ca.encode(y='Confirmed').properties(title='Total Confirmed') | base_ca.encode(y='Deaths',color = red).properties(title='Total deaths')
base_ca.encode(y='New cases').properties(title='CA State- New cases')

alt.Chart(...)

In [ ]:
#base_ny.encode(y='Confirmed').properties(title='Total Confirmed') | base_ny.encode(y='Deaths',color = red).properties(title='Total deaths')

base_ny.encode(y='New cases').properties(title='NY state - New cases')


alt.Chart(...)

# Export data to csv

In [ ]:
ca_df = full_grouped[full_grouped['Province_State'] == 'California']
ny_df = full_grouped[full_grouped['Province_State'] == 'New York']
tx_df = full_grouped[full_grouped['Province_State'] == 'Texas']

In [ ]:
ny_df.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
0,2020-01-22,New York,36001.0,0,0.0,0,0
1,2020-01-23,New York,36001.0,0,0.0,0,0


In [ ]:
ny_df.to_csv('NY-CovidDec26.csv',index=False)
ca_df.to_csv('CA-CovidDec26.csv',index=False)

# Compute rolling Average for new cases 

In [ ]:
ny_total.head()

,Date,Province_State,New cases
0,2020-01-22,New York,0
1,2020-01-23,New York,0
2,2020-01-24,New York,0
3,2020-01-25,New York,0
4,2020-01-26,New York,0


In [ ]:

ny_total['rolling_average'] = ny_total.iloc[:,2].rolling(window=7).mean()

base_ny_total = alt.Chart(ny_total).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)

In [ ]:
base_ny_total.encode(y='rolling_average').properties(title='NY state - rolling_average')


alt.Chart(...)

## Reference

* https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6